#### Import Libraries

In [3]:
import pandas as pd
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import openai
import pandas as pd
import numpy as np
import yaml
import os
import openai
from openai.embeddings_utils import cosine_similarity
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import swifter
from configparser import ConfigParser
from langchain.llms import OpenAI
import langchain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import GoogleSerperAPIWrapper
import os
import pprint
import json

#### Install Packages

In [4]:
# pip install chromadb
# pip install tiktoken


#### API KEY

In [49]:
api_key = 'Your API Key'
os.environ['OPENAI_API_KEY'] = api_key
os.environ["SERPER_API_KEY"] = "SERPER API KEY"

#### Utiliy Functions

In [6]:
def unique_list(a):
    '''
    Input:
    List
    Output:
    List
    Returns only unique values of a list
    '''
    unique = []
    for i in a:
        if i not in unique:
            unique.append(i)
    return unique

def dataframe_subset(df, column_list):
    '''
    Input:
    Dataframe and column list, consisting of columns to be subsetted
    
    Output:
    Dataframe
    
    Subsets dataframe basis of column names
    '''
    return df[column_list]

def dataframe_to_dictionary(df):
    '''
    Input:
    Dataframe
    Ouput:
    Dictionary
    Transforms the dataframe to dictionary
    '''
    try:
        a= df.to_dict(orient='records')
        return a
    except :
        b = df.to_dict()
        return b

def dictionary_subset(original_dictionary, keys_to_extract):
    '''
    Input:
    Dictionary
    Output:
    Dictionary
    Returns the dictionary with relevant keys

    A partciular example will have category, brand_name, flavour data for same sku, this will help to extract only relevant keys.
    '''
    return  {key: original_dictionary[key] for key in keys_to_extract if key in original_dictionary}


def list_dictionary_subset(original_list, keys_to_extract):
    '''
    Extracting different dictionaries from a list
    '''
    return [dictionary_subset(i,keys_to_extract) for i in original_list ]


def dictionary_to_example(input_dictionary, s = "To help you understand the answer I want, here are some similar examples below "):
    '''
    Converting a dictionary to a string format for examples set.
    '''
    for i in input_dictionary:
        s = s + "\n" + str(i).replace("{","").replace("}","").replace("'","")
    s = s + "\n"
    return s

def example_string(similar_examples, keys_to_extract):
    # dictionary key subset
    similar_examples_sub = list_dictionary_subset(similar_examples, keys_to_extract)

    # dictionary to example
    examples_string = dictionary_to_example(similar_examples_sub)

    return examples_string

def prompt_template_string(similar_examples, keys_to_extract,input_prompt, output_format_prompt, use_examples =  True ):

    '''
    Input:
    input_prompt - First part of the prompt which gives the question to the prompt

    examples - The input data which is used to be used as examples for example dataset
 
    sku_input - The input sku, for which the answer to be found
    
    keys_to_extract - The relevant key names in the examples dataset
    
    output_format_prompt - The output format of the prompt
    
    use_examples -  To keep examples in prompt or not
    
    k - the number of similar examples
    '''

    if use_examples == False:
        return input_prompt + output_format_prompt

    # prompt + examples

    examples_string = example_string(similar_examples= similar_examples, keys_to_extract= keys_to_extract)

    final_prompt = input_prompt + examples_string + output_format_prompt

    return final_prompt

def prompt_template(input_variables, template ):
    return PromptTemplate( template = template, input_variables=input_variables)

def semantic_similar_example(examples, sku_input, k = 5):
    '''
    Input
    examples - The list from where the examples to be choosen from
    sku_input - The user input sku
    k -  the maximum number of examples to return
    '''
    example_selector = SemanticSimilarityExampleSelector.from_examples(examples, OpenAIEmbeddings(model = 'text-embedding-ada-002'), Chroma, k = k)
    selected_examples = example_selector.select_examples({"SKU": sku_input})
    return unique_list(selected_examples)

    return d

def parse_json_values(data):
    '''
    Input - data - A dictionary where the values are dictionary within a string
    Output - A json file is returned 

    '''
    try:
        data = json.loads(data)
    except:
        data = data

    parsed_data = {}
    
    for key, value in data.items():
        try:
            parsed_value = json.loads(value)
            parsed_data[key] = parsed_value
        except json.JSONDecodeError:
            parsed_data[key] = value
    
    return parsed_data




#### Import Dataset

In [7]:
import pandas as pd

dataset_path = r"C:\Users\66660\Downloads\Everrise_Snacks_Sample_v2.xlsx"

example_path = r"C:\Users\66660\Downloads\snacks_initial_examples.xlsx"

initial_examples_df = pd.read_excel(example_path)

initial_examples_df['Category'] = initial_examples_df['Category'].str[7:]
initial_examples_df['Description'] = initial_examples_df['Description'].str[13:]

df = pd.read_excel(dataset_path)


# SKU Cleaning - Removing the barcode
df['Description'] = df['Description'].str[13:]
df['Category'] = df['Category'].str[7:]

#fill na

df = df.fillna("Not Applicable")


# Renaming columns
df.rename(columns = {'Description':'SKU', 'Brand' : 'brand' , 'Flavour' : 'flavour', 'Category' : 'category' , 'Pack size' : 'pack_size', 'Type' : 'sub_category', 'Price tier' : 'price_tier'}, inplace = True)

initial_examples_df.rename(columns = {'Description':'SKU', 'Brand' : 'brand' , 'Flavour' : 'flavour', 'Category' : 'category' , 'Pack size' : 'pack_size', 'Type' : 'sub_category', 'Price tier' : 'price_tier'}, inplace = True)


df = dataframe_subset(df, ['SKU', 'category', 'sub_category' , 'flavour' , 'brand' , 'pack_size'])

df.head()

C:\Users\66660\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,SKU,category,sub_category,flavour,brand,pack_size
0,KOREAN COLOR CURRY POPCORN 80G,CORN SNACKS,Not Applicable,Not Applicable,Not Applicable,Small
1,TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G,NUTS,Seeds,Not Applicable,Tong Garden,Medium
2,WHITE PUMPKIN SEED 500G,NUTS,Seeds,Not Applicable,Unbranded,Large
3,TAISUN TREATZ VEGGIE STICKS 250G,NUTS,Mixed nuts,Not Applicable,Taisun,Large
4,ALWI KACANG CORNFLAKES 40G,NUTS,Kacang,Not Applicable,Alwi,Small


In [8]:
print("The number of unique categories in the dataframe are",df.category.nunique())


The number of unique categories in the dataframe are 13


#### Example Creation

In [43]:
initial_examples = dataframe_to_dictionary(initial_examples_df)

initial_examples_df.head(2)

,category,sub_category,flavour,brand,SKU,pack_size
0,NUTS,Peanut,Cracker,Camel,CAMEL CRACKER PEANUT 40G,Small
1,NUTS,Peanut,Cracker,Tong Garden,TONG GARDEN PEANUT & CRACKER 50G,Small


#### Similarity Function Working

In [48]:
# Trying outputwith SKU for level 1
similar_examples = semantic_similar_example(examples= initial_examples, sku_input= sku_input,k=1)
similar_examples

[{'SKU': 'TAISUN TREATZ VEGGIE STICKS 250G',
  'brand_name': 'Taisun',
  'category': 'VEGETABLE SNACKS',
  'flavour': 'Not Found',
  'pack_size': '250G',
  'sub_category': 'Veggie Sticks'}]

#### Prompt Input

##### Prompt Level 1 Input

In [10]:
prompt_level1 =  """You are an expert in understanding Retail SKUs. You have an understanding across different industries and able to create Consumer Decision Trees.
Answer the category for this given product description: {SKU}."""
output_prompt_level1 = '''Also provide the confiedence score for your answer. Keep your confidence score as an integer between 0 to 10.
Return your output as a json with three keys - category, category_confidence_score. Your answer shouldn't be snacks.
ONLY RETURN THE JSON IN THE CORRECT FORMAT. Strictly review your output.'''

keys_to_extract_1 = ["SKU" , "category"]
output_key_1 = 'category'

##### level output

In [11]:
def level_output(sku_input, output_key , keys_to_extract = keys_to_extract_1,  input_variables = ['SKU'] , examples= initial_examples, input_prompt= prompt_level1, model = 'gpt-4-0613', temperature = 0.6,  output_format_prompt= output_prompt_level1):
    '''
    Input

    sku_input - Input string (string)

    keys_to_extract -  Relevant key names to get output from examples

    examples - dictionary to be used as input set

    input_prompt - The first part of the prompt

    output_format_prompt - The second part of the prompt

    Return

    Output for for any individual level 

    '''

    #llm define
    llm = ChatOpenAI(model = model, temperature = temperature)
    # Finding similar examples
    similar_examples = semantic_similar_example(examples= examples, sku_input= sku_input, k = 5)
    # The complete prompt string
    template_string = prompt_template_string(similar_examples,keys_to_extract= keys_to_extract, input_prompt= input_prompt, output_format_prompt= output_format_prompt)
    # complete prompt string in the given prompt_template
    prompt_l1 = prompt_template(input_variables= input_variables, template= prompt_template_string(similar_examples,keys_to_extract= keys_to_extract, input_prompt= input_prompt, output_format_prompt= output_format_prompt))

    level1_chain = LLMChain(llm=llm, prompt= prompt_l1, output_key= output_key)

    return level1_chain.run(SKU = sku_input)




In [12]:
# Trying outputwith SKU for level 1

a = level_output(sku_input= "SIM JAPANESE GREEN PEAS 300G", keys_to_extract= keys_to_extract_1, output_key= output_key_1)

a

'{"category": "NUTS", "category_confidence_score": 8}'

##### Prompt Level 2 Input

In [13]:
old_prompt_level2 = '''You are an expert in understanding Retails SKUs. 
Please find out the brand name or comapny name, from the sku and category information mentioned below
sku: {SKU} 
category: {category}
Here are some useful examples to help you make your decision'''

old_output_format_prompt_l2 = '''
Return only the Brand Name and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.'''

prompt_level2 =  """You are an expert in reading product desciptions and finding out the brand name.
Answer the brand name  or company name only using the description: {SKU}, answer the Brand Name or company of it, if it is not immediately clear from the description then use your best judgement to infer you can try using category name: {category}."""

output_prompt_level2 = '''Also provide the confiedence score and reason for your answer. Keep your confidence score as an integer between 0 to 10.
Return your output as a json with two keys - brand_name, brand_name_confidence_score.
ONLY RETURN THE JSON IN THE CORRECT FORMAT. Strictly review your output.'''

##### Pompt Level 3 Input

In [14]:
old_prompt_level3 = '''You are an expert in understanding Retails SKUs. 
What is the pack size for the SKU : {SKU}, category is {category} and brand name is {brand_name}.
Here are some useful examples to help you make your decision
'''

old_output_format_prompt_l3 = '''Note that, different categories might have different pack sizes. Your output should be between Small, Medium, Large, Not Applicable. Return only the output and nothing else. 
Strictly review your output.'''

prompt_level3 =  """You are an expert in understanding Retail SKUs. You have an understanding across different industries and able to create Consumer Decision Trees.
Extract the pack size from the description: {SKU} beonging to category: {category} belonging to brand {brand_name}.  """
output_prompt_level3 = '''Also provide the confiedence score. Keep your confidence score as an integer between 0 to 10.
Note that, different categories might have different pack sizes. If you're unable to extract pack size from the decription respond Not Applicable.
Return your output as a json, with two keys, pack_size, pack_size_confidence_score.
ONLY RETURN THE JSON IN THE CORRECT FORMAT.'''

##### Prompt Level 4 Input

In [15]:
old_prompt_level4 = '''You are an expert in understanding Retails SKUs. 
For a given snack which belongs to category: {category} , made by this brand name: {brand_name}, available in pack size: {pack_size}.
What is the flavour of snack with SKU: {SKU}?
Here are some examples to help you understand -
'''

old_output_format_prompt_l4 = '''
Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.'''

prompt_level4 =  """You are an expert in understanding Retail SKUs. You have an understanding across different industries and able to create Consumer Decision Trees.
For a given snack which belongs to category: {category} , made by this brand name: {brand_name}.
Try to infer the flavour of snack with SKU: {SKU}, if you are unable to infer it from the description then use your best judgement."""

output_prompt_level4 = '''Also provide the confiedence score. Keep your confidence score as an integer between 0 to 10.
Return your output as a json, with two keys, flavour, flavour_confidence_score.
If you are unable to find the flavour respond Not Found. ONLY RETURN THE JSON IN THE CORRECT FORMAT.'''

##### Prompt Level 5 Input

In [16]:
old_prompt_level5 = '''You are an expert in understanding Retails SKUs. 
What would be the sub_category of snack for SKU: {SKU}.
Given the category of the snack is {category} , made by brand/company {brand_name} available in pack size {pack_size}  
Here are some examples for you to understand: 
'''

old_output_format_prompt_l5 = '''
Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.'''

prompt_level5 =  """You are an expert in understanding Retail SKUs. You have an understanding across different industries and able to create Consumer Decision Trees.
For a given snack which belongs to category: {category} , of this brand name: {brand_name}.
Try to infer the sub_category of snack with SKU: {SKU}, if you are unable to infer it from the description then use your best judgement.
Your sub_category answer shouldn't have any of the following words {category}, {flavour}.
"""
output_prompt_level5 = '''Also provide the confiedence score. Keep your confidence score as an integer between 0 to 10.
If you are unable to find the answer respond Not Found. Strictly review your output. 
Your output should always be a dictionary, with two keys, sub_category, sub_category_confidence_score. ONLY RETURN THE JSON IN THE CORRECT FORMAT.'''

#### Example Pompt Output Level 1-5

In [17]:
llm = ChatOpenAI(model = 'gpt-4-0613', temperature = .7)
sku_input = 'MR MURUKU MIXTURE 150G'

similar_examples = semantic_similar_example(examples= initial_examples, sku_input= sku_input, k = 5)

print("Prompt - Category \n")

print(prompt_template_string(similar_examples,keys_to_extract= ['SKU','category'], input_prompt= prompt_level1, output_format_prompt= output_prompt_level1))

print("\n Prompt - Brand \n")

print(prompt_template_string(similar_examples, input_prompt= prompt_level2, output_format_prompt= output_prompt_level2, keys_to_extract = ["SKU", "category" , "brand"]))

print("\n Prompt - Pack Size \n")

print(prompt_template_string(similar_examples, input_prompt= prompt_level3, use_examples= False, output_format_prompt= output_prompt_level3, keys_to_extract = ["SKU", "category" , "brand", "pack_size"]))

print("\n Prompt - Flavour \n")
print(prompt_template_string(similar_examples, input_prompt= prompt_level4, output_format_prompt= output_prompt_level4, keys_to_extract = ["SKU", "category" , "brand", "pack_size", 'flavour']))

print("\n Prompt - Sub_Category \n")
print(prompt_template_string(similar_examples, input_prompt= prompt_level5, output_format_prompt= output_prompt_level5, keys_to_extract = ["SKU", "category" , "brand", "pack_size", 'flavour', 'sub_category']))

Prompt - Category 

You are an expert in understanding Retail SKUs. You have an understanding across different industries and able to create Consumer Decision Trees.
Answer the category for this given product description: {SKU}.To help you understand the answer I want, here are some similar examples below 
SKU: ROYAL FAMILY MILK TEA JAPANESE MOCHI 120G, category: OTHERS
SKU: TRIKO SHJ CRISPY ROLL COFFEE FLR 165G, category: OTHERS
SKU: TONG GARDEN PEANUT & CRACKER 50G, category: NUTS
Also provide the confiedence score for your answer. Keep your confidence score as an integer between 0 to 10.
Return your output as a json with three keys - category, category_confidence_score. Your answer shouldn't be snacks.
ONLY RETURN THE JSON IN THE CORRECT FORMAT. Strictly review your output.

 Prompt - Brand 

You are an expert in reading product desciptions and finding out the brand name.
Answer the brand name  or company name only using the description: {SKU}, answer the Brand Name or company of it

#### Sequential Chain Function

In [18]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain



def sequential_chain_output(sku_input, examples = initial_examples , prompt_level1 = prompt_level1 , output_prompt_l1 = output_prompt_level1, prompt_level2 = prompt_level2,  output_prompt_l2 = output_prompt_level2 , prompt_level3 = prompt_level3, output_prompt_l3 = output_prompt_level3,prompt_level4 = prompt_level4, output_prompt_l4 = output_prompt_level4, prompt_level5 = prompt_level5, output_prompt_l5 = output_prompt_level5,   output_key_l1 = "category", output_key_l2 = "brand_name", output_key_l3 = "pack_size",output_key_l4 = "flavour", output_key_l5 = 'sub_category' , keys_to_extract_l1 = ["SKU", "category"], keys_to_extract_l2 = ["SKU", "category" , "brand"], keys_to_extract_l3 = ["SKU", "category" , "brand", "pack_size"], keys_to_extract_l4 = ["SKU", "category" , "brand", "pack_size", 'flavour'], keys_to_extract_l5 = ["SKU", "category" , "brand",  'sub_category', 'flavour'] ):

    llm = ChatOpenAI(model = 'gpt-4-0613', temperature= .7, openai_api_key = api_key)
 
    similar_examples = semantic_similar_example(examples= examples, sku_input= sku_input, k = 5)
   
    # Prompts from all the functions 
    prompt_l1 = prompt_template(input_variables= ['SKU'], template= prompt_template_string(similar_examples,keys_to_extract= keys_to_extract_l1, input_prompt= prompt_level1, output_format_prompt= output_prompt_l1))
    prompt_l2 = prompt_template(input_variables = ['SKU', 'category'], template= prompt_template_string(similar_examples,keys_to_extract= keys_to_extract_l2, input_prompt= prompt_level2, output_format_prompt= output_prompt_l2))
    prompt_l3 = prompt_template(input_variables = ['SKU', 'category', 'brand_name'], template= prompt_template_string(similar_examples,keys_to_extract= keys_to_extract_l3, input_prompt= prompt_level3, use_examples= False, output_format_prompt= output_prompt_l3))
    prompt_l4 = prompt_template(input_variables = ['SKU', 'category', 'brand_name'], template= prompt_template_string(similar_examples,keys_to_extract= keys_to_extract_l4, input_prompt= prompt_level4, output_format_prompt= output_prompt_l4))
    prompt_l5 = prompt_template(input_variables=['SKU', 'category', 'brand_name' , 'flavour'],template= prompt_template_string(similar_examples,keys_to_extract= keys_to_extract_l5, input_prompt= prompt_level5, output_format_prompt= output_prompt_l5))

    # Individual level chains
    level1_chain = LLMChain(llm=llm, prompt= prompt_l1, output_key= output_key_l1)
    level2_chain = LLMChain(llm=llm, prompt= prompt_l2, output_key= output_key_l2)
    level3_chain = LLMChain(llm=llm, prompt= prompt_l3, output_key= output_key_l3)
    level4_chain = LLMChain(llm=llm, prompt= prompt_l4, output_key= output_key_l4)
    level5_chain = LLMChain(llm=llm, prompt= prompt_l5, output_key= output_key_l5)

    overall_chain = SequentialChain(chains=[level1_chain, level2_chain, level3_chain, level4_chain, level5_chain], input_variables=["SKU"], output_variables=['category' ,"brand_name",'pack_size','flavour','sub_category'], verbose=True)
    chain_response = overall_chain({"SKU" : sku_input})
    return chain_response


# print(a[1])

#### Example: Sequential Chain Output 

In [19]:
a = sequential_chain_output(sku_input = "MISTER POTATO BBQ 75G")
a



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'MISTER POTATO BBQ 75G',
 'category': '{"category": "POTATOES CHIPS", "category_confidence_score": 10}',
 'brand_name': '{"brand_name": "Mister Potato", "brand_name_confidence_score": 10}',
 'pack_size': '{\n"pack_size": "75G",\n"pack_size_confidence_score": 10\n}',
 'flavour': '{\n"flavour": "BBQ",\n"flavour_confidence_score": 10\n}',
 'sub_category': '{"sub_category": "Not Available", "sub_category_confidence_score": 10}'}

In [20]:
def additional_example(json_input):
    '''
    Input - llm chain output - We extract the category, brand_name, pack_size, sub_category and flavour
    Output - we extract relevant information from llm chain output to added to the examples
    '''
    json_input = parse_json_values(json_input)
    d = {}
    keys = ['category', 'brand_name', 'pack_size' , 'flavour', 'sub_category']
    d['SKU'] = json_input['SKU']
    for i in keys:
      d[i] = json_input[i][i]
    return d



#### Addtional Example Output example

In [21]:
additional_example(a)

{'SKU': 'MISTER POTATO BBQ 75G',
 'category': 'POTATOES CHIPS',
 'brand_name': 'Mister Potato',
 'pack_size': '75G',
 'flavour': 'BBQ',
 'sub_category': 'Not Available'}

#### Langchain Apply Function

In [22]:
# initital_examples = dataframe_to_dictionary(initial_examples_df)
# example_pc_nuts = dataframe_to_dictionary(pc_nuts.head(12))


def sequential_chain_output_adding_examples(sku_input):

    '''
    Getting sequential chain output and adding example to initial examples
    '''

    try:
        answer = sequential_chain_output(sku_input = sku_input)
        initial_examples.append(additional_example(answer))
    except: 
        answer = sequential_chain_output(sku_input = sku_input)
        print(answer)
    return answer

def apply_langchain_function(df, description):

    '''
    Applying the sequential langchain function in every row.
    '''
    df['langchain_sol1'] = df[description].swifter.apply(lambda x : sequential_chain_output_adding_examples(x))
    return df

def parse_langchain_output(result):
    'Parse json of the langchain solution for differnt column names'

    result = parse_json_values(result)
    try :
       category = result['category']['category']
       category_cs = result['category']['category_confidence_score']
    except:
        category = result['category']
        category_cs = result['category']
    try:
        brand_name = result['brand_name']['brand_name']
        brand_name_cs = result['brand_name']['brand_name_confidence_score']
    except:
        brand_name = result['brand_name']
        brand_name_cs = result['brand_name']        
    try:
        pack_size = result['pack_size']['pack_size']
        pack_size_cs = result['pack_size']['pack_size_confidence_score']
    except:
        brand_name = result['brand_name']
        brand_name_cs = result['brand_name']
    try:
        flavour =  result['flavour']['flavour']
        flavour_cs =  result['flavour']['flavour_confidence_score']
    except:
        flavour =  result['flavour']
        flavour_cs =  result['flavour']
    try:
        sub_category = result['sub_category']['sub_category']
        sub_category_cs = result['sub_category']['sub_category_confidence_score']
    except:
        sub_category = result['sub_category']
        sub_category_cs = result['sub_category']

    return category, category_cs, brand_name, brand_name_cs, pack_size, pack_size_cs, flavour, flavour_cs, sub_category, sub_category_cs

def apply_parse_langchain_output(df):
    '''
    For this problem specific creating columns relevant for problem in hand
    '''
    df['lc_category'], df['lc_category_cs'], df['lc_brand_name'], df['lc_brand_name_cs'], df['lc_pack_size'], df['lc_pack_size_cs'], df['lc_flavour'], df['lc_flavour_cs'], df['lc_sub_category'], df['lc_sub_category_cs'] = zip(*df['langchain_sol1'].apply(parse_langchain_output))
    return df

In [23]:
def final_langchain_sequential_output(input_data, description):

    '''
    input_data - Input Dataset
    description - The column name with SKU description

    Return
    The final output with langchain output
    '''

    # Step 1 -  creating langchain output column as json file

    intermediate_df = apply_langchain_function(input_data, description)

    # step 2 parsing the json column into different column output

    final_output = apply_parse_langchain_output(intermediate_df)

    return final_output

#### Sequential Chain Output Example

In [24]:
test_5 = df.head(5)
test_5_results = final_langchain_sequential_output(test_5, 'SKU')

Pandas Apply:   0%|          | 0/5 [00:00<?, ?it/s]



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'WHITE PUMPKIN SEED 500G', 'category': '{"category": "NUTS", "category_confidence_score": 9}', 'brand_name': '{"brand_name": "Unknown", "brand_name_confidence_score": 0}', 'pack_size': '{\n"pack_size": "500G",\n"pack_size_confidence_score": 10\n}', 'flavour': '{\n"flavour": "Not Found",\n"flavour_confidence_score": 0\n}', 'sub_category': '{"sub_category": "Pumpkin Seeds", "sub_category_confidence_score": 8}'}


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


C:\Users\66660\AppData\Local\Temp\ipykernel_25448\3180203768.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['langchain_sol1'] = df[description].swifter.apply(lambda x : sequential_chain_output_adding_examples(x))
C:\Users\66660\AppData\Local\Temp\ipykernel_25448\3180203768.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lc_category'], df['lc_category_cs'], df['lc_brand_name'], df['lc_brand_name_cs'], df['lc_pack_size'], df['lc_pack_size_cs'], df['lc_flavour'], df['lc_flavour_cs'], df['lc_s

In [25]:
test_5_results

,SKU,category,sub_category,flavour,brand,pack_size,langchain_sol1,lc_category,lc_category_cs,lc_brand_name,lc_brand_name_cs,lc_pack_size,lc_pack_size_cs,lc_flavour,lc_flavour_cs,lc_sub_category,lc_sub_category_cs
0,KOREAN COLOR CURRY POPCORN 80G,CORN SNACKS,Not Applicable,Not Applicable,Not Applicable,Small,"{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'cat...",POPCORN,8,Korean Color Curry,8,80G,10,Not Found,0,Curry,7
1,TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G,NUTS,Seeds,Not Applicable,Tong Garden,Medium,{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL ...,NUTS,9,TONG GARDEN,10,130G,10,Not Found,0,Sunflower Seeds,10
2,WHITE PUMPKIN SEED 500G,NUTS,Seeds,Not Applicable,Unbranded,Large,"{'SKU': 'WHITE PUMPKIN SEED 500G', 'category':...",NUTS,9,Unknown,0,500G,10,Not Found,0,Pumpkin Seeds,8
3,TAISUN TREATZ VEGGIE STICKS 250G,NUTS,Mixed nuts,Not Applicable,Taisun,Large,"{'SKU': 'TAISUN TREATZ VEGGIE STICKS 250G', 'c...",VEGETABLE SNACKS,9,Taisun,10,250G,10,Not Found,0,Veggie Sticks,8
4,ALWI KACANG CORNFLAKES 40G,NUTS,Kacang,Not Applicable,Alwi,Small,"{'SKU': 'ALWI KACANG CORNFLAKES 40G', 'categor...",NUTS,8,ALWI,7,40G,9,Cornflakes,7,Kacang,7


#### Google Agent Prompt

In [27]:
initial_prompt_agent = ''' For the following product description tell me the Category, Brand Name , Flavour and pack size, description :
'''

output_prompt_agent = '''If you don't have an answer for any of the questions, respond as not applicabe. Your response should ALWAYS be a json file with the following dictionary keys google_category, google_brand_name, google_flavour, google_pack_size. Don't return anything but a json file. 
'''

#### Google Agent Function

In [29]:
def agent_google_search(sku_input, initial_prompt = initial_prompt_agent, output_prompt = output_prompt_agent, verbose = False):
    '''
    Input:
    initial_prompt (string) : The starting part of the prompt
    sku_input (string) : The SKU description to search for
    output_prompt (string) : The last part of the prompt to describe the output format

    The initial_prompt + sku_input + output_prompt is concatenated as string and given as an input

    verbose :  Boolean, whether to return the intermediate thought process or not

    Output:
    Answer in a json format.
    '''

    llm = OpenAI(temperature=.6)
    search = GoogleSerperAPIWrapper()
    tools = [
    Tool(
        name="Google Search",
        func=search.run,
        description="useful to search for anykind of information available in the internet",
    )]
    self_ask_with_search = initialize_agent(tools, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose  =  verbose)
    return self_ask_with_search.run(initial_prompt+sku_input+output_prompt)

def apply_agent_google_search(df, description):
    df['google_agent_json_soln'] = df[description].swifter.apply(lambda x : agent_google_search(x))
    return df

#### Google Agent Apply Function

In [33]:
def parse_google_agent_output(result):
    'Parse json of the langchain solution for differnt column names'
    #print(result)
    try:
        result = json.loads(result)
        category = result['google_category']
        brand_name = result['google_brand_name']
        pack_size = result['google_pack_size']
        flavour =  result['google_flavour']
    except:
        category = ''
        brand_name = ''
        pack_size = ''
        flavour = ''
    return category, brand_name, pack_size, flavour

def apply_parse_google_agent_output(df):
    df['google_category'], df['google_brand_name'], df['google_pack_size'], df['google_flavour'] = zip(*df['google_agent_json_soln'].apply(parse_google_agent_output))
    return df

In [34]:
def final_google_agent_output(input_data, description):

    '''
    input_data - Input Dataset
    description - The column name with SKU description

    Return
    The final output with google search agent output
    '''

    # Step 1 -  creating langchain output column as json file

    intermediate_df = apply_agent_google_search(input_data, description)

    # step 2 parsing the json column into different column output

    final_output = apply_parse_google_agent_output(intermediate_df)

    return final_output

#### Example Google Agent Apply Function 

In [35]:
test_5 = df.head(5)
final_google_agent_output(test_5,'SKU')

Pandas Apply:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_25448\1424949499.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google_agent_json_soln'] = df[description].swifter.apply(lambda x : agent_google_search(x))
C:\Users\66660\AppData\Local\Temp\ipykernel_25448\118964832.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google_category'], df['google_brand_name'], df['google_pack_size'], df['google_flavour'] = zip(*df['google_agent_json_soln'].apply(parse_google_agent_output))
C:\Users\66660\AppData\Loc

,SKU,category,sub_category,flavour,brand,pack_size,google_agent_json_soln,google_category,google_brand_name,google_pack_size,google_flavour
0,KOREAN COLOR CURRY POPCORN 80G,CORN SNACKS,Not Applicable,Not Applicable,Not Applicable,Small,"{\n ""google_category"": ""Food Delivery"",\n ...",Food Delivery,Korean Colors,80g,Spicy Curry
1,TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G,NUTS,Seeds,Not Applicable,Tong Garden,Medium,"{'google_category': 'Snack', 'google_brand_nam...",,,,
2,WHITE PUMPKIN SEED 500G,NUTS,Seeds,Not Applicable,Unbranded,Large,"{\n ""google_category"": ""Pumpkin Seeds"",\n ""g...",Pumpkin Seeds,OUYANGHENGZHI,500g/17.6oz,Original Taste Nanguazi 原味南瓜子
3,TAISUN TREATZ VEGGIE STICKS 250G,NUTS,Mixed nuts,Not Applicable,Taisun,Large,"{\n ""google_category"": ""Veggie Sticks"",\n ...",Veggie Sticks,Taisun,250g,Veggie
4,ALWI KACANG CORNFLAKES 40G,NUTS,Kacang,Not Applicable,Alwi,Small,"{\n ""google_category"": ""Snacks"",\n ""goog...",Snacks,Combywide,40g,Kacang Snap


#### Final Output Function

In [36]:
def final_output_function(input_data,description_column_name , option):
    if option == 'Langchain':
        output = final_langchain_sequential_output(input_data= input_data, description= description_column_name)
    elif option == 'GoogleAgent':
        output = final_google_agent_output(input_data= input_data, description = description_column_name)
    elif option == 'Both':
        intermediate_output = final_langchain_sequential_output(input_data= input_data, description= description_column_name)
        output = final_google_agent_output(input_data= intermediate_output, description = description_column_name)
    else:
        return 'Please use a valid options among any of the folloing options - Langchain, Google Agent, Both '
    
    return output

#### Final Output Function Examples

In [38]:
test_5 = df.head(5)

final_output_function(test_5, 'SKU', option= 'Langchain')

Pandas Apply:   0%|          | 0/5 [00:00<?, ?it/s]



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'category': '{"category": "POPCORN", "category_confidence_score": 9}', 'brand_name': '{"brand_name": "Korean Color Curry", "brand_name_confidence_score": 7}', 'pack_size': '{\n"pack_size": "80G",\n"pack_size_confidence_score": 10\n}', 'flavour': '{"flavour": "Not Found", "flavour_confidence_score": 0}', 'sub_category': '{"sub_category": "Curry", "sub_category_confidence_score": 7}'}


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'WHITE PUMPKIN SEED 500G', 'category': '{"category": "NUTS", "category_confidence_score": 9}', 'brand_name': 'The provided information does not give a clear brand name or company name for the product WHITE PUMPKIN SEED 500G. Therefore, I cannot accurately inf

C:\Users\66660\AppData\Local\Temp\ipykernel_25448\3180203768.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['langchain_sol1'] = df[description].swifter.apply(lambda x : sequential_chain_output_adding_examples(x))
C:\Users\66660\AppData\Local\Temp\ipykernel_25448\3180203768.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lc_category'], df['lc_category_cs'], df['lc_brand_name'], df['lc_brand_name_cs'], df['lc_pack_size'], df['lc_pack_size_cs'], df['lc_flavour'], df['lc_flavour_cs'], df['lc_s

,SKU,category,sub_category,flavour,brand,pack_size,langchain_sol1,lc_category,lc_category_cs,lc_brand_name,lc_brand_name_cs,lc_pack_size,lc_pack_size_cs,lc_flavour,lc_flavour_cs,lc_sub_category,lc_sub_category_cs
0,KOREAN COLOR CURRY POPCORN 80G,CORN SNACKS,Not Applicable,Not Applicable,Not Applicable,Small,"{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'cat...",POPCORN,9,Korean Color Curry,7,80G,10,Not Found,0,Curry,7
1,TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G,NUTS,Seeds,Not Applicable,Tong Garden,Medium,{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL ...,NUTS,10,TONG GARDEN,10,130G,10,Not Found,10,Sunflower Seeds,10
2,WHITE PUMPKIN SEED 500G,NUTS,Seeds,Not Applicable,Unbranded,Large,"{'SKU': 'WHITE PUMPKIN SEED 500G', 'category':...",NUTS,9,The provided information does not give a clear...,The provided information does not give a clear...,500G,10,Not Found,0,Pumpkin Seeds,9
3,TAISUN TREATZ VEGGIE STICKS 250G,NUTS,Mixed nuts,Not Applicable,Taisun,Large,"{'SKU': 'TAISUN TREATZ VEGGIE STICKS 250G', 'c...",VEGETABLE SNACKS,10,TAISUN,10,250G,10,Not Found,0,Veggie Sticks,10
4,ALWI KACANG CORNFLAKES 40G,NUTS,Kacang,Not Applicable,Alwi,Small,"{'SKU': 'ALWI KACANG CORNFLAKES 40G', 'categor...",NUTS,9,ALWI,9,40G,9,Cornflakes,9,Kacang,9


In [39]:
test_5 = df.head(5)
final_output_function(test_5, 'SKU', option= 'GoogleAgent')

Pandas Apply:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_25448\1424949499.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google_agent_json_soln'] = df[description].swifter.apply(lambda x : agent_google_search(x))
C:\Users\66660\AppData\Local\Temp\ipykernel_25448\118964832.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google_category'], df['google_brand_name'], df['google_pack_size'], df['google_flavour'] = zip(*df['google_agent_json_soln'].apply(parse_google_agent_output))
C:\Users\66660\AppData\Loc

,SKU,category,sub_category,flavour,brand,pack_size,google_agent_json_soln,google_category,google_brand_name,google_pack_size,google_flavour
0,KOREAN COLOR CURRY POPCORN 80G,CORN SNACKS,Not Applicable,Not Applicable,Not Applicable,Small,"{\n ""google_category"": ""Snacks"",\n ""goog...",Snacks,Korean Colors,80 grams,Spicy Curry Mushroom
1,TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G,NUTS,Seeds,Not Applicable,Tong Garden,Medium,"{\n ""google_category"": ""Snack"",\n ""googl...",Snack,Tong Garden,130g,Salted
2,WHITE PUMPKIN SEED 500G,NUTS,Seeds,Not Applicable,Unbranded,Large,"{\n ""google_category"": ""Pumpkin Seeds"",\n ""g...",Pumpkin Seeds,OUYANGHENGZHI,500g/17.6oz,Original Taste Nanguazi 原味南瓜子
3,TAISUN TREATZ VEGGIE STICKS 250G,NUTS,Mixed nuts,Not Applicable,Taisun,Large,"{\n ""google_category"": ""Snacks"",\n ""goog...",Snacks,TAISUN,250G,Chilli
4,ALWI KACANG CORNFLAKES 40G,NUTS,Kacang,Not Applicable,Alwi,Small,"{\n ""google_category"": ""Not Applicable"",\n ...",Not Applicable,ALWI,40G,KACANG CORNFLAKES


In [40]:
test_5 = df.head(5)
final_output_function(test_5, 'SKU', option= 'Both')

Pandas Apply:   0%|          | 0/5 [00:00<?, ?it/s]



> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'category': '{"category": "POPCORN", "category_confidence_score": 10}', 'brand_name': '{"brand_name": "Unknown", "brand_name_confidence_score": 0}', 'pack_size': '{\n"pack_size": "80G",\n"pack_size_confidence_score": 10\n}', 'flavour': '{\n"flavour": "Curry",\n"flavour_confidence_score": 8\n}', 'sub_category': '{"sub_category": "Korean Snacks", "sub_category_confidence_score": 7}'}


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.


C:\Users\66660\AppData\Local\Temp\ipykernel_25448\3180203768.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['langchain_sol1'] = df[description].swifter.apply(lambda x : sequential_chain_output_adding_examples(x))
C:\Users\66660\AppData\Local\Temp\ipykernel_25448\3180203768.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lc_category'], df['lc_category_cs'], df['lc_brand_name'], df['lc_brand_name_cs'], df['lc_pack_size'], df['lc_pack_size_cs'], df['lc_flavour'], df['lc_flavour_cs'], df['lc_s

Pandas Apply:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_25448\1424949499.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google_agent_json_soln'] = df[description].swifter.apply(lambda x : agent_google_search(x))
C:\Users\66660\AppData\Local\Temp\ipykernel_25448\118964832.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google_category'], df['google_brand_name'], df['google_pack_size'], df['google_flavour'] = zip(*df['google_agent_json_soln'].apply(parse_google_agent_output))
C:\Users\66660\AppData\Loc

,SKU,category,sub_category,flavour,brand,pack_size,langchain_sol1,lc_category,lc_category_cs,lc_brand_name,...,lc_pack_size_cs,lc_flavour,lc_flavour_cs,lc_sub_category,lc_sub_category_cs,google_agent_json_soln,google_category,google_brand_name,google_pack_size,google_flavour
0,KOREAN COLOR CURRY POPCORN 80G,CORN SNACKS,Not Applicable,Not Applicable,Not Applicable,Small,"{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'cat...",POPCORN,10,Unknown,...,10,Curry,8,Korean Snacks,7,"{\n ""google_category"": ""Snack"",\n ""googl...",Snack,Korean Colors,80 grams,Spicy Curry Mushroom
1,TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G,NUTS,Seeds,Not Applicable,Tong Garden,Medium,{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL ...,NUTS,9,Tong Garden,...,10,Not Found,0,Sunflower Seeds,10,"{\n ""google_category"": ""Snack"",\n ""google_br...",Snack,Tong Garden,130G,Barbecue
2,WHITE PUMPKIN SEED 500G,NUTS,Seeds,Not Applicable,Unbranded,Large,"{'SKU': 'WHITE PUMPKIN SEED 500G', 'category':...",NUTS,9,Unknown,...,10,Not Found,0,Pumpkin Seeds,9,"{\n ""google_category"": ""Seeds"",\n ""googl...",Seeds,OUYANGHENGZHI,500g/17.6oz * 2 bags = 1kg/35.27oz,"Original Taste Nanguazi 原味南瓜子, Huamei Taste"
3,TAISUN TREATZ VEGGIE STICKS 250G,NUTS,Mixed nuts,Not Applicable,Taisun,Large,"{'SKU': 'TAISUN TREATZ VEGGIE STICKS 250G', 'c...",VEGETABLE SNACKS,9,Taisun,...,10,Not Found,0,Veggie Sticks,10,"{\n ""google_category"": ""Veggie Sticks"",\n ...",Veggie Sticks,Taisun,250g,BBQ/Chilli
4,ALWI KACANG CORNFLAKES 40G,NUTS,Kacang,Not Applicable,Alwi,Small,"{'SKU': 'ALWI KACANG CORNFLAKES 40G', 'categor...",NUTS,8,ALWI,...,10,Cornflakes,9,Kacang,7,"{\n ""google_category"": ""Not Applicable"",\n ...",Not Applicable,Combywide,150g,Kacang Snap
